In [4]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os
import json
from tqdm import tqdm

os.environ['OPENAI_API_KEY']="" #OPENAI API KEY를 넣으시면 됩니다 
openai.api_key = os.getenv("OPENAI_API_KEY")

하단은 Open AI API 코드로 IT관련되어 12개 카테고리에 맞춰 시리즈 파일을 받으면 분류하는 코드(가장 저렴한 3.5 터보엔진 활용)

In [23]:

# 환경 변수에서 API 키 로드
openai.api_key = os.getenv("OPENAI_API_KEY")

# 분류할 카테고리 목록
CATEGORIES = [
    "백엔드 개발자", "프론트엔드 개발자", "iOS 개발자", "풀스택 개발자",
    "모바일 앱 개발자", "데이터 엔지니어/데이터 과학자", "AI/ML 엔지니어",
    "DevOps/SRE 엔지니어", "QA 엔지니어", "보안/정보보호", "PM", "기타"
]

def classify_job(job_text):
    """
    직업 텍스트를 12개 카테고리 중 하나로 분류 (결과에서 '카테고리:' 등 제거)
    """
    try:
        system_prompt = (
            f"다음은 직업 텍스트를 주어진 카테고리 중 하나로 정확히 분류하는 작업입니다.\n"
            f"카테고리 목록: {', '.join(CATEGORIES)}\n"
            f"직업 텍스트가 모호할 경우, 가장 유사한 카테고리를 선택하거나 '기타'를 반환하세요."
        )
        
        # OpenAI API 호출
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"직업 텍스트: {job_text}"},
            ],            max_tokens=100,
            temperature=0.3,  # 낮은 온도로 결정론적 응답 생성
        )
        # 응답 내용 추출 및 정리
        result = response["choices"][0]["message"]["content"].strip()
        
        # "카테고리:" 같은 불필요한 텍스트 제거
        for category in CATEGORIES:
            if category in result:
                return category
        return "기타"  # 카테고리 매칭 실패 시 기본값
    except Exception as e:
        return f"Error: {str(e)}"

def classify_job_list_with_progress(job_list):
    """
    직업 텍스트의 리스트를 받아 게이지 진행 표시와 함께 카테고리 분류
    """
    results = []
    for job_text in tqdm(job_list, desc="직업 분류 진행 중"):
        results.append(classify_job(job_text))
    return results

def classify_job_series_with_progress(job_series):
    """
    Pandas 시리즈 형태의 직업 텍스트를 받아 게이지 진행 표시와 함께 카테고리 분류
    """
    return job_series.progress_apply(classify_job)

In [26]:
import pandas as pd

job_series = pd.Series(df['classified_series'][:5])
classified_series = classify_job_list_with_progress(job_series)

직업 분류 진행 중: 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.63it/s]


In [11]:
df = pd.read_excel('c://python//최종본2.xlsx') 